# 用python发邮件
#### [https://zhuanlan.zhihu.com/p/57252226](https://zhuanlan.zhihu.com/p/57252226)

### 安全连接
#### SMTP_SSL() & .starttls()
* Gmail的一个不错的功能是，你可以使用 + 号在@符号前面的电子邮件地址中添加任何修饰符。 例如，发送到 my+person1@gmail.com 和 my+person2@gmail.com 的邮件都将发送到 my@gmail.com 。 在测试电子邮件功能时，你可以使用此功能模拟所有指向同一收件箱的多个地址。
* 你可以通过在命令提示符下输入以下内容来启动本地SMTP调试服务器：
* python -m smtpd -c DebuggingServer -n localhost:1025

#### 发送纯文本邮件
* 选项1: 使用 SMPT_SSL()
##### 使用 with smtplib.SMTP_SSL() as server 确保连接在缩进代码块的末尾自动关闭。 如果port为零或未指定，则 .SMTP_SSL() 将使用标准端口（端口465）。
##### 将电子邮件密码存储在代码中并不安全，特别是如果你打算与他人共享。 相反，使用input() 让用户在运行脚本时输入密码，如上例所示。 如果你在输入时不希望密码显示在屏幕上，则可以导入 getpass 模块并使用 .getpass() 代替直接输入密码。

In [2]:
# import smtplib, ssl

# port = 465  # For SSL
# password = input("Type your password and press enter: ")

# # Create a secure SSL context
# context = ssl.create_default_context()

# with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
#     server.login("my@gmail.com", password)
#     # TODO: Send email here

* 选项2: 使用 .starttls()
##### 要通知服务器让它知道自己，应在创建 .SMTP() 对象后调用 .helo() （SMTP）或 .ehlo() （ESMTP），并在 .starttls() 后再调用。 此函数由 .starttls() 和 .sendmail() 隐式调用，因此除非你要检查服务器的SMTP服务扩展，否则不必显式调用 .helo() 或 .ehlo() 

In [1]:
import smtplib, ssl

smtp_server = "smtp.gmail.com"
port = 587  # For starttls
sender_email = "my@gmail.com"
password = input("Type your password and press enter: ")
receiver_email = "your@gmail.com"
message = """\
Subject: app test

This message is sent from Python."""

# Create a secure SSL context
context = ssl.create_default_context()

# Try to log in to server and send email。为了确保再出现问题时代码不会崩溃，将主代码放到try中，让except将任何错误消息打印到 stdout
try:
    server = smtplib.SMTP(smtp_server,port)
    server.ehlo() # Can be omitted
    server.starttls(context=context) # Secure the connection
    server.ehlo() # Can be omitted
    server.login(sender_email, password)
    # TODO: Send email here
except Exception as e:
    # Print any error messages to stdout
    print(e)
finally:
    server.quit()

Type your password and press enter: odtxybaggivhsmle
(535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials r188sm4463579pfr.16 - gsmtp')


### 发送纯文本邮件

In [ ]:
server.sendmail(sender_email, receiver_email, message)

#### SMTP_SSL() 发送纯文本电邮实例

In [4]:
import smtplib, ssl

port = 465 #for SSL
smtp_server = "smtp.gmail.com"
sender_email = "olivia19931019@gmail.com"
receiver_email = "yanlaile1019@gmail.com"
password = input("type your password: ")
message = """\
Subject: App testing

This message is sent from Python."""

context = ssl.create_default_context()
with smtplib.SMTP_SSL(smtp_server, port, context = context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message)

type your password: rnqbtscrqmjzcvfz


#### .starttls() 发送纯文本电邮实例

In [2]:
import smtplib, ssl

port = 587 # for starttls
smtp_server = "smtp.gmail.com"
sender_email = "yanlaile1019@gmail.com"
receiver_email = "olivia19931019@gmail.com"
password = input("type your password: ")
message = """\
Subject: Hi App

Love never fails!"""

context = ssl.create_default_context()
with smtplib.SMTP(smtp_server, port) as server:
    server.ehlo()
    server.starttls(context = context)
    server.ehlo()
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message)

type your password: odtxybaggivhsmle


## 发送非纯文本邮件

### 包含HTML内容
* 常见电邮类型是MIME(多用途因特网邮件扩展)，它结合了HTML和纯文本；MIME消息由Python的email.mime模块处理
* 由于并非所有电子邮件客户端都默认显示HTML内容，并且出于安全原因，有些人仅选择接收纯文本电子邮件，因此为HTML消息添加纯文本的替代非常重要。 由于电子邮件客户端将首先渲染最后一部分的附件，因此请确保在纯文本版本之后添加HTML消息。
* MIMEMultipart("alternative")实例将这些组合成一个带有两个备用渲染选项的消息

In [9]:
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

sender_email = "olivia19931019@gmail.com"
receiver_email = "olivia19931019@gmail.com"
password = input("Type your pwd: ")

message = MIMEMultipart("alternative")
message['Subject'] = "multipart test"
message['From'] = sender_email
message["To"] = receiver_email

# create the plain-text and HTML version of your message
text = """\
Hi,
How are you?
Real Python has many great tutorials:
www.realpython.com"""
html = """\
<html>
    <body>
        <p>Hi,<br>
            How are you?<br>
            <a href = "http://www.realpython.com">Real Python</a>
            has many great tutorials.
        </p>
    </body>
</html>
"""

# Turn these into plain/html MIMEText objects
part1 = MIMEText(text, "plain")
part2 = MIMEText(html, "html")

# Add HTML/plain-text parts to MIMEMultipart message
# The email client will try to render the last part first
message.attach(part1)
message.attach(part2)

# Create secure connection with server and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context = context) as server:
    server.login(sender_email, password)
    server.sendmail(
    sender_email, receiver_email, message.as_string()
    )

Type your pwd: Zhous_8875


### 使用email添加附件
##### 为了将二进制文件发送到设计用于处理文本数据的电子邮件服务器，需要在传输之前对其进行编码。 这通常使用base64完成，它将二进制数据编码为可打印的ASCII字符。
* 将pdf文件作为附件的实例
* MIMEultipart() 接受RFC5233样式的键/值对形式的参数，这些参数存储在字典中并传递给 Message 基类的 .add_header方法。
* 查看Python的 email.mime 模块的文档，了解有关使用MIME类的更多信息。

In [13]:
import email, smtplib, ssl

from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

subject = "An email with pdf attached from python"
body = "This is an email with attached sent from Python"
sender_email = "olivia19931019@gmail.com"
receiver_email = "olivia19931019@gmail.com"
password = input("type: ")

# Create a multipart message and set headers
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = subject
message["Bcc"] = receiver_email # recommended for mass emails

# Add body to email
message.attach(MIMEText(body, "plain"))

filename = "document.pdf" # in same directory as script

# open PDF file in binary mode
with open(filename, "rb") as attachment:
    # Add file as application/octet-stream
    # Email client can usually download this automatically as attachment
    part = MIMEBase("application", "octet-stream")
    part.set_payload(attachment.read())
    
# Encode file in ASCII characters to send by email
encoders.encode_base64(part)

# Add header as key/value pair to attachment part
part.add_header(
    "Content-Disposition",
    f"attachment; filename = {filename}",
)

# Add attachment to message and convert message to string
message.attach(part)
text = message.as_string()

# Log in to server using secure context and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context = context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, text)

type: Zhous_8875


### 发送多封个性化电子邮件